In [141]:
import IPython
import numpy as np
import tvm
from tvm.ir.module import IRModule
from tvm.script import tir as T
from matplotlib import pyplot as plt 
import math 
from tvm.script import tir 

2.5. Exercises for TensorIR¶

2.4.5.1: Exercise using j_factor

In [142]:
@tvm.script.ir_module
class MyModule:
    @T.prim_func
    def mm_relu(A: T.Buffer[(128, 128), "float32"],
                B: T.Buffer[(128, 128), "float32"],
                C: T.Buffer[(128, 128), "float32"]):
        T.func_attr({"global_symbol": "mm_relu", "tir.noalias": True})
        Y = T.alloc_buffer((128, 128), dtype="float32")
        for i, j, k in T.grid(128, 128, 128):
            with T.block("Y"):
                vi = T.axis.spatial(128, i)
                vj = T.axis.spatial(128, j)
                vk = T.axis.reduce(128, k)
                with T.init():
                    Y[vi, vj] = T.float32(0)
                Y[vi, vj] = Y[vi, vj] + A[vi, vk] * B[vk, vj]
        for i, j in T.grid(128, 128):
            with T.block("C"):
                vi = T.axis.spatial(128, i)
                vj = T.axis.spatial(128, j)
                C[vi, vj] = T.max(Y[vi, vj], T.float32(0))

In [143]:
def transform(mod, jfactor):
    sch = tvm.tir.Schedule(mod)
    block_Y = sch.get_block("Y", func_name="mm_relu")
    i, j, k = sch.get_loops(block_Y)
    j0, j1 = sch.split(j, factors=[None, jfactor])
    sch.reorder(j0, k, j1)
    block_C = sch.get_block("C", "mm_relu")
    sch.reverse_compute_at(block_C, j0)
    return sch.mod

def try_with_jfactor(jfactor: int):
    dtype = "float32"
    a_np = np.random.rand(128, 128).astype(dtype)
    b_np = np.random.rand(128, 128).astype(dtype)
    a_nd = tvm.nd.array(a_np)
    b_nd = tvm.nd.array(b_np)
    c_nd = tvm.nd.empty((128, 128), dtype="float32")
    type(c_nd)

    mod_transformed = transform(MyModule, jfactor=jfactor)

    rt_lib_transformed = tvm.build(mod_transformed, "llvm")
    f_timer_transformed = rt_lib_transformed.time_evaluator("mm_relu", tvm.cpu())
    return f_timer_transformed(a_nd, b_nd, c_nd).mean, mod_transformed

In [144]:
mean_benchmarks = []
jfactors = []

for jfactor in range(1, 129):
    times = []
    for repeats in range(10):
        time, mod_transformed = try_with_jfactor(jfactor)
        times.append(time)
    mean_benchmark = sum(times) / len(times)
    print(f"jfactor {jfactor}: {mean_benchmark} (s)")
    mean_benchmarks.append(mean_benchmark)
    jfactors.append(jfactor)

log_mean_benchmarks = [math.log(d) for d in mean_benchmarks]
plt.scatter(jfactors, log_mean_benchmarks)

log_mean_benchmarks = [(log_mean_benchmarks[i], i + 1) for i in range(len(log_mean_benchmarks))]
log_mean_benchmarks = sorted(log_mean_benchmarks)
print(log_mean_benchmarks)

# 22 jfactor, 128, and 64 are the fastest! Weird that 22 is so fast...
# May also be imperfect breaking of the loops 
# Also shape of graph is weird... perhaps related to some kind of codegen thing?

jfactor 1: 0.00413448124 (s)
jfactor 2: 0.0007935012499999999 (s)
jfactor 3: 0.00068636875 (s)
jfactor 4: 0.00042908458 (s)
jfactor 5: 0.00045317416999999995 (s)
jfactor 6: 0.00041749585 (s)
jfactor 7: 0.0004534608400000001 (s)
jfactor 8: 0.00026729375 (s)
jfactor 9: 0.00045268666 (s)
jfactor 10: 0.00024596919 (s)
jfactor 11: 0.00029234834 (s)
jfactor 12: 0.00021513289 (s)
jfactor 13: 0.00021232376000000002 (s)
jfactor 14: 0.00026907833 (s)
jfactor 15: 0.00030697209 (s)
jfactor 16: 0.00039187414999999995 (s)
jfactor 17: 0.00030614876999999997 (s)
jfactor 18: 0.00026941626 (s)
jfactor 19: 0.00021991001 (s)


KeyboardInterrupt: 

In [61]:
_, mod_22 = try_with_jfactor(22)
_, mod_23 = try_with_jfactor(23)
_, mod_21 = try_with_jfactor(21)
_, mod_20 = try_with_jfactor(20)
_, mod_128 = try_with_jfactor(128)
_, mod_127 = try_with_jfactor(127)
_, mod_31 = try_with_jfactor(31)

print(f"22\n{mod_22.script()}\n\n")
print(f"23\n{mod_23.script()}\n\n")
print(f"21\n{mod_21.script()}\n\n")
print(f"20\n{mod_20.script()}\n\n")
print(f"128\n{mod_128.script()}\n\n")
print(f"127\n{mod_127.script()}\n\n")
print(f"31\n{mod_31.script()}\n\n")

22
# from tvm.script import tir as T
@tvm.script.ir_module
class Module:
    @T.prim_func
    def mm_relu(A: T.Buffer[(128, 128), "float32"], B: T.Buffer[(128, 128), "float32"], C: T.Buffer[(128, 128), "float32"]) -> None:
        # function attr dict
        T.func_attr({"global_symbol": "mm_relu", "tir.noalias": True})
        # body
        # with T.block("root")
        Y = T.alloc_buffer([128, 128], dtype="float32")
        for i, j_0 in T.grid(128, 6):
            for k, j_1 in T.grid(128, 22):
                with T.block("Y"):
                    vi = T.axis.spatial(128, i)
                    vj = T.axis.spatial(128, j_0 * 22 + j_1)
                    vk = T.axis.reduce(128, k)
                    T.where(j_0 * 22 + j_1 < 128)
                    T.reads(A[vi, vk], B[vk, vj])
                    T.writes(Y[vi, vj])
                    with T.init():
                        Y[vi, vj] = T.float32(0)
                    Y[vi, vj] = Y[vi, vj] + A[vi, vk] * B[vk, vj]
            f

2.5.1.1. Example: Element-wise Add

In [145]:
# init data
a = np.arange(16).reshape(4, 4)
b = np.arange(16, 0, -1).reshape(4, 4)

In [146]:
# numpy version
c_np = a + b
c_np

array([[16, 16, 16, 16],
       [16, 16, 16, 16],
       [16, 16, 16, 16],
       [16, 16, 16, 16]])

In [147]:
# low-level numpy version
def lnumpy_add(a: np.ndarray, b: np.ndarray, c: np.ndarray):
  for i in range(4):
    for j in range(4):
      c[i, j] = a[i, j] + b[i, j]
c_lnumpy = np.empty((4, 4), dtype=np.int64)
lnumpy_add(a, b, c_lnumpy)
c_lnumpy

array([[16, 16, 16, 16],
       [16, 16, 16, 16],
       [16, 16, 16, 16],
       [16, 16, 16, 16]])

In [148]:
# TensorIR version
@tvm.script.ir_module
class MyAdd:
  @T.prim_func
  def add(A: T.Buffer[(4, 4), "int64"],
          B: T.Buffer[(4, 4), "int64"],
          C: T.Buffer[(4, 4), "int64"]):
    T.func_attr({"global_symbol": "add"})
    for i, j in T.grid(4, 4):
      with T.block("C"):
        vi = T.axis.spatial(4, i)
        vj = T.axis.spatial(4, j)
        C[vi, vj] = A[vi, vj] + B[vi, vj]

rt_lib = tvm.build(MyAdd, target="llvm")
a_tvm = tvm.nd.array(a)
b_tvm = tvm.nd.array(b)
c_tvm = tvm.nd.array(np.empty((4, 4), dtype=np.int64))
rt_lib["add"](a_tvm, b_tvm, c_tvm)
np.testing.assert_allclose(c_tvm.numpy(), c_np, rtol=1e-5)

2.5.1.2. Exercise 1: Broadcast Add

In [149]:
# init data
a = np.arange(16).reshape(4, 4)
b = np.arange(4, 0, -1).reshape(4)

In [150]:
# numpy version
c_np = a + b
c_np

array([[ 4,  4,  4,  4],
       [ 8,  8,  8,  8],
       [12, 12, 12, 12],
       [16, 16, 16, 16]])

In [151]:
@tvm.script.ir_module
class MyAdd:
  @T.prim_func
  def add(
      A: T.Buffer[(4, 4), "int64"], 
      B: T.Buffer[(4), "int64"],
      C: T.Buffer[(4, 4), "int64"]
    ):
        T.func_attr({"global_symbol": "add", "tir.noalias": True})
        for i, j in T.grid(4, 4):
            with T.block("C"):
                vi = T.axis.spatial(4, i)
                vj = T.axis.spatial(4, j)
                C[vi, vj] = A[vi, vj] + B[vj]

rt_lib = tvm.build(MyAdd, target="llvm")
a_tvm = tvm.nd.array(a)
b_tvm = tvm.nd.array(b)
c_tvm = tvm.nd.array(np.empty((4, 4), dtype=np.int64))
rt_lib["add"](a_tvm, b_tvm, c_tvm)
np.testing.assert_allclose(c_tvm.numpy(), c_np, rtol=1e-5)

2.5.1.3. Exercise 2: 2D Convolution¶


In [152]:
N, CI, H, W, CO, K = 1, 1, 8, 8, 2, 3
OUT_H, OUT_W = H - K + 1, W - K + 1
data = np.arange(N*CI*H*W).reshape(N, CI, H, W)
weight = np.arange(CO*CI*K*K).reshape(CO, CI, K, K)

In [153]:
# torch version
import torch

data_torch = torch.Tensor(data)
weight_torch = torch.Tensor(weight)
conv_torch = torch.nn.functional.conv2d(data_torch, weight_torch)
conv_torch = conv_torch.numpy().astype(np.int64)
conv_torch

array([[[[             474,              510,              546,
                       582,              618,              654],
         [             762,              798,              834,
                       870,              906,              942],
         [            1050,             1086,             1122,
                      1158,             1194,             1230],
         [            1338,             1374,             1410,
                      1446,             1482,             1518],
         [            1626,             1662,             1698,
                      1734,             1770,             1806],
         [            1914,             1950,             1986,
                      2022,             2058,             2094]],

        [[               0,                0, 8951137046626304,
                         0,                0,                0],
         [               0,                0,                0,
                         0,    

In [154]:
@tvm.script.ir_module
class MyConv:
  N, CI, H, W, CO, K = 1, 1, 8, 8, 2, 3
  OUT_H, OUT_W = H - K + 1, W - K + 1
  @T.prim_func
  def conv(
    in_img: T.Buffer[(1, 1, 8, 8), "int64"],
    filters: T.Buffer[(2, 1, 3, 3), "int64"],
    out_img: T.Buffer[(1, 2, 6, 6), "int64"]
  ):
    T.func_attr({"global_symbol": "conv", "tir.noalias": True})
    for n, co, i, j, kh, kw in T.grid(1, 2, 6, 6, 3, 3):
      with T.block("conv"):
        vn, vco, vi, vj, vkh, vkw = T.axis.remap("SSSSRR", [n, co, i, j, kh, kw]) 
        with T.init():
          out_img[vn, vco, vi, vj] = T.int64(0)
        out_img[vn, vco, vi, vj] += in_img[0, 0, vi + vkh, vj + vkw] * filters[vco, 0, vkh, vkw]

In [155]:
rt_lib = tvm.build(MyConv, target="llvm")
data_tvm = tvm.nd.array(data)
weight_tvm = tvm.nd.array(weight)
conv_tvm = tvm.nd.array(np.empty((N, CO, OUT_H, OUT_W), dtype=np.int64))
rt_lib["conv"](data_tvm, weight_tvm, conv_tvm)
print(conv_tvm.numpy())
print(conv_torch)
np.testing.assert_allclose(conv_tvm.numpy(), conv_torch, rtol=1e-5)

[[[[ 474  510  546  582  618  654]
   [ 762  798  834  870  906  942]
   [1050 1086 1122 1158 1194 1230]
   [1338 1374 1410 1446 1482 1518]
   [1626 1662 1698 1734 1770 1806]
   [1914 1950 1986 2022 2058 2094]]

  [[1203 1320 1437 1554 1671 1788]
   [2139 2256 2373 2490 2607 2724]
   [3075 3192 3309 3426 3543 3660]
   [4011 4128 4245 4362 4479 4596]
   [4947 5064 5181 5298 5415 5532]
   [5883 6000 6117 6234 6351 6468]]]]
[[[[             474              510              546              582
                 618              654]
   [             762              798              834              870
                 906              942]
   [            1050             1086             1122             1158
                1194             1230]
   [            1338             1374             1410             1446
                1482             1518]
   [            1626             1662             1698             1734
                1770             1806]
   [            1914

AssertionError: 
Not equal to tolerance rtol=1e-05, atol=0

Mismatched elements: 36 / 72 (50%)
Max absolute difference: 8972506656404243
Max relative difference: 1.
 x: array([[[[ 474,  510,  546,  582,  618,  654],
         [ 762,  798,  834,  870,  906,  942],
         [1050, 1086, 1122, 1158, 1194, 1230],...
 y: array([[[[             474,              510,              546,
                       582,              618,              654],
         [             762,              798,              834,...

2.5.2.2 Batch Matmul

In [156]:
def lnumpy_mm_relu_v2(A: np.ndarray, B: np.ndarray, C: np.ndarray):
    Y = np.empty((16, 128, 128), dtype="float32")
    for n in range(16):
        for i in range(128):
            for j in range(128):
                for k in range(128):
                    if k == 0:
                        Y[n, i, j] = 0
                    Y[n, i, j] = Y[n, i, j] + A[n, i, k] * B[n, k, j]
    for n in range(16):
        for i in range(128):
            for j in range(128):
                C[n, i, j] = max(Y[n, i, j], 0)

In [157]:
@tvm.script.ir_module
class MyBmmRelu:
  @T.prim_func
  def bmm_relu(
      A: T.Buffer[(16, 128, 128), "float32"],
      B: T.Buffer[(16, 128, 128), "float32"],
      C: T.Buffer[(16, 128, 128), "float32"]
  ):
    T.func_attr({"global_symbol": "bmm_relu", "tir.noalias": True})

    intermediate = T.alloc_buffer((16, 128, 128), "float32")
    for n, i, j, k in T.grid(16, 128, 128, 128):
      with T.block("matmul"):
        vn, vi, vj, vk = T.axis.remap("SSSR", [n, i, j, k])
        with T.init(): 
          intermediate[vn, vi, vj] = T.float32(0)
        intermediate[vn, vi, vj] += A[vn, vi, vk] * B[vn, vk, vj]
    for n, i, j in T.grid(16, 128, 128):
      with T.block("relu"):
        vn, vi, vj = T.axis.remap("SSS", [n, i, j])
        C[vn, vi, vj] = T.max(intermediate[vn, vi, vj], T.float32(0))

print(MyBmmRelu.script())
#sch = tvm.tir.Schedule(MyBmmRelu)
#IPython.display.Code(sch.mod.script(), language="python")
# Also please validate your result

# from tvm.script import tir as T
@tvm.script.ir_module
class Module:
    @T.prim_func
    def bmm_relu(A: T.Buffer[(16, 128, 128), "float32"], B: T.Buffer[(16, 128, 128), "float32"], C: T.Buffer[(16, 128, 128), "float32"]) -> None:
        # function attr dict
        T.func_attr({"global_symbol": "bmm_relu", "tir.noalias": True})
        # body
        # with T.block("root")
        intermediate = T.alloc_buffer([16, 128, 128], dtype="float32")
        for n, i, j, k in T.grid(16, 128, 128, 128):
            with T.block("matmul"):
                vn, vi, vj, vk = T.axis.remap("SSSR", [n, i, j, k])
                T.reads(A[vn, vi, vk], B[vn, vk, vj])
                T.writes(intermediate[vn, vi, vj])
                with T.init():
                    intermediate[vn, vi, vj] = T.float32(0)
                intermediate[vn, vi, vj] = intermediate[vn, vi, vj] + A[vn, vi, vk] * B[vn, vk, vj]
        for n, i, j in T.grid(16, 128, 128):
            with T.block("relu"):
                v

In [179]:
sch = tvm.tir.Schedule(MyBmmRelu)
print("original")
print(sch.mod.script())

matmul_block = sch.get_block("matmul", func_name="bmm_relu")

relu_block = sch.get_block("relu", func_name="bmm_relu")
relu_n, relu_i, relu_j = sch.get_loops(relu_block)

relu_j0, relu_j1 = sch.split(relu_j, [16, 8])

sch.compute_at(matmul_block, relu_n)
sch.compute_at(matmul_block, relu_i)
sch.compute_at(matmul_block, relu_j0)

sch.parallel(relu_n)
sch.vectorize(relu_j1)

matmul_n, matmul_i, matmul_j0, matmul_ax0, matmul_ax1 = sch.get_loops(matmul_block)
init_block = sch.decompose_reduction(matmul_block, matmul_ax0)

sch.reorder(matmul_ax1, matmul_ax0)
ax1_0, ax1_1 = sch.split(matmul_ax1, [32, 4])
sch.unroll(ax1_1)
sch.vectorize(sch.get_loops(init_block)[3])


print("now")
print(sch.mod.script())
# print("target")
# print(tvm.tir.Schedule(TargetModule).mod.script())
# tvm.ir.assert_structural_equal(sch.mod, TargetModule)



original
# from tvm.script import tir as T
@tvm.script.ir_module
class Module:
    @T.prim_func
    def bmm_relu(A: T.Buffer[(16, 128, 128), "float32"], B: T.Buffer[(16, 128, 128), "float32"], C: T.Buffer[(16, 128, 128), "float32"]) -> None:
        # function attr dict
        T.func_attr({"global_symbol": "bmm_relu", "tir.noalias": True})
        # body
        # with T.block("root")
        intermediate = T.alloc_buffer([16, 128, 128], dtype="float32")
        for n, i, j, k in T.grid(16, 128, 128, 128):
            with T.block("matmul"):
                vn, vi, vj, vk = T.axis.remap("SSSR", [n, i, j, k])
                T.reads(A[vn, vi, vk], B[vn, vk, vj])
                T.writes(intermediate[vn, vi, vj])
                with T.init():
                    intermediate[vn, vi, vj] = T.float32(0)
                intermediate[vn, vi, vj] = intermediate[vn, vi, vj] + A[vn, vi, vk] * B[vn, vk, vj]
        for n, i, j in T.grid(16, 128, 128):
            with T.block("relu"):
        

NameError: name 'tir' is not defined